In [45]:
import numpy as np
import pandas as pd
import statistics
import math
import seaborn as sns
import matplotlib.pyplot as plt
import ast
import json

In [2]:
import escher
from escher import Builder
import cobra
from time import sleep

In [3]:
simData = np.load(r"../../out/geneRxnVerifData/output.npy", allow_pickle=True, encoding='ASCII')
fluxesWithCaption = simData.tolist()['agents']['0']['listeners']['fba_results']['estimated_fluxes']
complexes = simData.tolist()['agents']['0']['bulk']

ecData = pd.read_csv(r"../new genes/geneData.txt")
ecReactions = ecData["reactionID"].values.tolist()
temp = []
for word in ecReactions:
    if not word == "No Reaction Found":
        temp.append(word)
ecReactions = temp

metabolData = np.load(r"../../out/geneRxnVerifData/stoichiometry.npy", allow_pickle=True, encoding='ASCII')
rxn_metabolites = metabolData.tolist()

In [4]:
rxns = list(rxn_metabolites.keys())
glucoseRxns = {}
fructoseRxns = {}
for rxn in rxns:
    metabolites = rxn_metabolites[rxn].keys()
    for metabolite in metabolites:
        if metabolite == 'GLC[p]':
            glucoseRxns[rxn] = rxn_metabolites[rxn]
        if metabolite == 'FRU[p]':
            fructoseRxns[rxn] = rxn_metabolites[rxn]

In [5]:
fructoseRxns

{'TRANS-RXN0-582-FRU/PTSI-PHOSPHORYLATED//CPD-16154/PTSI-MONOMER.48.': {'FRU[p]': -1,
  'PTSI-PHOSPHORYLATED[c]': -1,
  'CPD-16154[c]': 1,
  'PTSI-MONOMER[c]': 1},
 'TRANS-RXN0-582-FRU/PTSI-PHOSPHORYLATED//FRU1P/PTSI-MONOMER.44.': {'FRU[p]': -1,
  'PTSI-PHOSPHORYLATED[c]': -1,
  'FRU1P[c]': 1,
  'PTSI-MONOMER[c]': 1}}

In [6]:
glucoseRxns

{'3.2.1.21-RXN-Beta-D-glucosides/WATER//Non-Glucosylated-Glucose-Acceptors/GLC.64.': {'Beta-D-glucosides[p]': -1,
  'WATER[p]': -1,
  'Non-Glucosylated-Glucose-Acceptors[p]': 1,
  'GLC[p]': 1},
 'GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//GLC/Pi.36.': {'GLC-1-P[p]': -1,
  'WATER[p]': -1,
  'GLC[p]': 1,
  'Pi[p]': 1},
 'RXN0-6373-GLC/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.41.': {'GLC[p]': -1,
  'UBIQUINONE-8[i]': -1,
  'GLC-D-LACTONE[p]': 1,
  'CPD-9956[i]': 1},
 'RXN0-6418': {'GLC[p]': -1, 'D-Glucose[p]': 1},
 'RXN0-6418 (reverse)': {'GLC[p]': 1, 'D-Glucose[p]': -1},
 'RXN0-6418-GLC//CPD-15374.15.': {'GLC[p]': -1, 'CPD-15374[p]': 1},
 'RXN0-6418-GLC//CPD-15374.15. (reverse)': {'GLC[p]': 1, 'CPD-15374[p]': -1},
 'RXN0-7077-GLC/PROTON//ALPHA-GLUCOSE/PROTON.33.': {'GLC[p]': -1,
  'PROTON[p]': -1,
  'ALPHA-GLUCOSE[c]': 1,
  'PROTON[c]': 1},
 'RXN0-7077-GLC/PROTON//GLC/PROTON.23.': {'GLC[p]': -1,
  'PROTON[p]': -1,
  'GLC[c]': 1,
  'PROTON[c]': 1},
 'RXN0-7077-GLC/PROTON//Glucopyrano

In [7]:
fluxes = {}
for rKeys in fructoseRxns.keys():
    fluxes[rKeys] = fluxesWithCaption[rKeys]
fluxes

{'TRANS-RXN0-582-FRU/PTSI-PHOSPHORYLATED//CPD-16154/PTSI-MONOMER.48.': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'TRANS-RXN0-582-FRU/PTSI-PHOSPHORYLATED//FRU1P/PTSI-MONOMER.44.': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [8]:
fluxes2 = {}
for rKeys2 in glucoseRxns.keys():
    fluxes2[rKeys2] = fluxesWithCaption[rKeys2]
fluxes2

{'3.2.1.21-RXN-Beta-D-glucosides/WATER//Non-Glucosylated-Glucose-Acceptors/GLC.64.': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'GLUCOSE-1-PHOSPHAT-RXN[CCO-PERI-BAC]-GLC-1-P/WATER//GLC/Pi.36.': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'RXN0-6373-GLC/UBIQUINONE-8//GLC-D-LACTONE/CPD-9956.41.': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'RXN0-6418': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'RXN0-6418 (reverse)': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 

In [9]:
glycolysis = ['PGLUCISOM-RXN', '6PFRUCTPHOS-RXN', 'F16BDEPHOS-RXN', 'F16ALDOLASE-RXN', 'TRIOSEPISOMERIZATION-RXN', 'GAPOXNPHOSPHN-RXN', 'PHOSGLYPHOS-RXN', 'RXN-15513', '3PGAREARR-RXN', '2PGADEHYDRAT-RXN', 'PEPDEPHOS-RXN', 'PEPSYNTH-RXN']

In [10]:
glycolysis

['PGLUCISOM-RXN',
 '6PFRUCTPHOS-RXN',
 'F16BDEPHOS-RXN',
 'F16ALDOLASE-RXN',
 'TRIOSEPISOMERIZATION-RXN',
 'GAPOXNPHOSPHN-RXN',
 'PHOSGLYPHOS-RXN',
 'RXN-15513',
 '3PGAREARR-RXN',
 '2PGADEHYDRAT-RXN',
 'PEPDEPHOS-RXN',
 'PEPSYNTH-RXN']

In [11]:
glycolysisFluxes = {}
for rxn in glycolysis:
    if rxn in fluxesWithCaption.keys():
        glycolysisFluxes[rxn] = fluxesWithCaption[rxn]
    else:
        glycolysisFluxes[rxn] = "Flux Data Not Found"

In [12]:
glycolysisFluxes

{'PGLUCISOM-RXN': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 '6PFRUCTPHOS-RXN': 'Flux Data Not Found',
 'F16BDEPHOS-RXN': 'Flux Data Not Found',
 'F16ALDOLASE-RXN': 'Flux Data Not Found',
 'TRIOSEPISOMERIZATION-RXN': [71584,
  63052,
  76101,
  72203,
  75704,
  71729,
  83500,
  71414,
  75717,
  84937,
  91304,
  89963,
  87695,
  93845,
  86897,
  94217,
  96747,
  98343,
  92132,
  102526,
  96443,
  93545,
  98601,
  102631,
  96630,
  98590,
  94321,
  103106,
  99856,
  107701],
 'GAPOXNPHOSPHN-RXN': [156054,
  175047,
  174906,
  178208,
  171227,
  177638,
  166379,
  182675,
  182994,
  175950,
  179295,
  184866,
  176644,
  179096,
  184465,
  174872,
  183336,
  184081,
  189032,
  175140,
  186104,
  180829,
  174709,
  178690,
  190286,
  181755,
  181339,
  178355,
  186539,
  177895],
 'PHOSGLYPHOS-RXN': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [13]:
notFound = ['6PFRUCTPHOS-RXN', 'F16BDEPHOS-RXN', 'F16ALDOLASE-RXN', 'PEPDEPHOS-RXN']

foundNames = []
for rxn in notFound:
    for key in fluxesWithCaption:
        if rxn in key or key in rxn:
            foundNames.append((rxn, key))

In [14]:
foundNames

[('6PFRUCTPHOS-RXN', '6PFRUCTPHOS-RXN__6PFK-1-CPX'),
 ('6PFRUCTPHOS-RXN', '6PFRUCTPHOS-RXN__6PFK-2-CPX'),
 ('F16BDEPHOS-RXN',
  'F16BDEPHOS-RXN[CCO-CYTOSOL]-FRUCTOSE-16-DIPHOSPHATE/WATER//FRUCTOSE-6P/Pi.59.__CPLX0-303'),
 ('F16BDEPHOS-RXN',
  'F16BDEPHOS-RXN[CCO-CYTOSOL]-FRUCTOSE-16-DIPHOSPHATE/WATER//FRUCTOSE-6P/Pi.59.__CPLX0-7776'),
 ('F16BDEPHOS-RXN',
  'F16BDEPHOS-RXN[CCO-CYTOSOL]-FRUCTOSE-16-DIPHOSPHATE/WATER//FRUCTOSE-6P/Pi.59.__EG11239-MONOMER'),
 ('F16BDEPHOS-RXN',
  'F16BDEPHOS-RXN[CCO-CYTOSOL]-FRUCTOSE-16-DIPHOSPHATE/WATER//FRUCTOSE-6P/Pi.59.__F16B-CPLX'),
 ('F16BDEPHOS-RXN',
  'F16BDEPHOS-RXN[CCO-PERI-BAC]-FRUCTOSE-16-DIPHOSPHATE/WATER//FRUCTOSE-6P/Pi.60.'),
 ('F16ALDOLASE-RXN', 'F16ALDOLASE-RXN (reverse)'),
 ('F16ALDOLASE-RXN', 'F16ALDOLASE-RXN__FRUCBISALD-CLASSI'),
 ('F16ALDOLASE-RXN', 'F16ALDOLASE-RXN__FRUCBISALD-CLASSII'),
 ('PEPDEPHOS-RXN', 'PEPDEPHOS-RXN (reverse)')]

In [15]:
for x in foundNames:
    print(x[0], fluxesWithCaption[x[1]])

6PFRUCTPHOS-RXN [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6PFRUCTPHOS-RXN [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
F16BDEPHOS-RXN [732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732, 732]
F16BDEPHOS-RXN [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
F16BDEPHOS-RXN [1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1802, 1796, 1796, 1796]
F16BDEPHOS-RXN [7432, 7432, 7472, 7472, 7472, 7472, 7472, 7472, 7472, 7472, 7472, 7472, 7472, 7513, 7513, 7513, 7513, 7513, 7513, 7513, 7513, 7554, 7554, 7554, 7554, 7554, 7594, 7594, 7594, 7594]
F16BDEPHOS-RXN [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
F16ALDOLASE-RXN 

In [16]:
prev = ""
for names in foundNames:
    if prev == names[0]:
        for i in range(len(glycolysisFluxes[prev])):
            glycolysisFluxes[prev][i] += fluxesWithCaption[names[1]][i]
    else:
        prev = names[0]
        glycolysisFluxes[prev] = fluxesWithCaption[names[1]]

In [17]:
glycolysisFluxes

{'PGLUCISOM-RXN': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 '6PFRUCTPHOS-RXN': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'F16BDEPHOS-RXN': [9970,
  9970,
  10010,
  10010,
  10010,
  10010,
  10010,
  10010,
  10010,
  10010,
  10010,
  10010,
  10010,
  10051,
  10051,
  10051,
  10051,
  10051,
  10051,
  10051,
  10051,
  10092,
  10092,
  10092,
  10092,
  10092,
  10132,
  10126,
  10126,
  10126],
 'F16ALDOLASE-RXN': [157576,
  157576,
  157943,
  157985,
  158122,
  158156,
  158371,
  158450,
  158590,
  158707,
  158837,
  158928,
  159126,
  159291,
  159393,
  159522,
  159644,
  159702,
  159829,
  159956,
  160142,
  160278,
  160350,
  160395,
  160509,
  160558,
  160637,
  160712,
  160815,
  160869],
 'TRIOSEPISOMERIZATION-RXN': [71584,
  63052,
  76101,


In [37]:
fluxLast = []

for key in glycolysisFluxes.keys():
    fluxes = glycolysisFluxes[key]
    fluxLast.append(fluxes[-1])

In [38]:
fluxLast

[0, 0, 10126, 160869, 107701, 177895, 0, 0, 0, 177268, 0, 445971]

In [39]:
escher.list_available_maps()

[{'organism': 'Saccharomyces cerevisiae',
  'map_name': 'iMM904.Central carbon metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Inositol retinol metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Glycolysis TCA PPP'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Tryptophan metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Carbohydrate metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Amino acid metabolism (partial)'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Nucleotide metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Nucleotide and histidine biosynthesis'},
 {'organism': 'Escherichia coli', 'map_name': 'e_coli_core.Core metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid beta-oxidation'}

In [40]:
escher.list_available_models()

[{'organism': 'Saccharomyces cerevisiae', 'model_name': 'iMM904'},
 {'organism': 'Homo sapiens', 'model_name': 'RECON1'},
 {'organism': 'Escherichia coli', 'model_name': 'e_coli_core'},
 {'organism': 'Escherichia coli', 'model_name': 'iJO1366'}]

In [41]:
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
    reaction_data={r: v for r, v in zip(glycolysisFluxes.keys(), fluxLast)},
)

In [46]:
builder

Builder(allow_building_duplicate_reactions=False, and_method_in_gene_reaction_rule='mean', cofactors=['atp', '…

In [51]:
with open('../../iJO1366.Central metabolism.json') as json_file:
    data = json.load(json_file)

In [59]:
escherRxns = data[1]['reactions']
names = []

for number in escherRxns:
    reactionName = escherRxns[number]['name']
    names.append(reactionName)

In [67]:
names

['Malate dehydrogenase',
 'Nitrite Reductase (Ubiquinole-8, periplasm)',
 'NADPH Quinone Reductase (2-Demethylmenaquinone-8)',
 'NADH dehydrogenase (menaquinone-8 & 3 protons) (periplasm)',
 'L-malate transport out via proton antiport (periplasm)',
 'D-lactate transport via proton symport (periplasm)',
 'Glucose-1-phosphatase',
 'Trans-aconitate methyltransferase',
 'Acetaldehyde transport via diffusion (extracellular to periplasm)',
 'Glycolate oxidase',
 'Aconitate isomerase (spontaneous)',
 'Cytochrome oxidase bd (menaquinol-8: 2 protons) (periplasm)',
 'Phosphoenolpyruvate carboxylase',
 'Fructose 6-phosphate transport via diffusion (extracellular to periplasm)',
 'Malate synthase',
 'Hydrogenase (Demethylmenaquinone-8: 2 protons) (periplasm)',
 'Sedoheptulose 1,7-bisphosphate D-glyceraldehyde-3-phosphate-lyase',
 'Ethanol reversible transport via diffusion (periplasm)',
 'Cytochrome oxidase bd (ubiquinol-8: 2 protons) (periplasm)',
 'Fumarate transport via proton symport (2 H) (pe

In [63]:
enzymeNames = ['glucose-6-phosphate isomerase', '6-phosphofructokinase', 'fructose-bisphosphatase', 'fructose-bisphosphate aldolase', 'triose-phosphate isomerase', 'glyceraldehyde-3-phosphate dehydrogenase (phosphorylating)', 'phosphoglycerate kinase', 'pyruvate kinase', 'phosphoglycerate mutase (2,3-diphosphoglycerate-dependent)', 'pyruvate-water dikinase (phosphorylating)', 'phosphopyruvate hydratase']

In [64]:
enzymeNames

['glucose-6-phosphate isomerase',
 '6-phosphofructokinase',
 'fructose-bisphosphatase',
 'fructose-bisphosphate aldolase',
 'triose-phosphate isomerase',
 'glyceraldehyde-3-phosphate dehydrogenase (phosphorylating)',
 'phosphoglycerate kinase',
 'pyruvate kinase',
 'phosphoglycerate mutase (2,3-diphosphoglycerate-dependent)',
 'pyruvate-water dikinase (phosphorylating)',
 'phosphopyruvate hydratase']

In [68]:
glycolysis

['PGLUCISOM-RXN',
 '6PFRUCTPHOS-RXN',
 'F16BDEPHOS-RXN',
 'F16ALDOLASE-RXN',
 'TRIOSEPISOMERIZATION-RXN',
 'GAPOXNPHOSPHN-RXN',
 'PHOSGLYPHOS-RXN',
 'RXN-15513',
 '3PGAREARR-RXN',
 '2PGADEHYDRAT-RXN',
 'PEPDEPHOS-RXN',
 'PEPSYNTH-RXN']

In [72]:
want = []
for rxn in enzymeNames:
    for name in names:
        if name.lower() in rxn.lower() or rxn.lower() in name.lower():
            want.append((rxn, name))

In [ ]:
nameAssociation = {
    'PGLUCISOM-RXN': 'glucose-6-phosphate isomerase',
    '6PFRUCTPHOS-RXN': 'Phosphofructokinase',
    'fructose-bisphosphatase

In [73]:
want

[('glucose-6-phosphate isomerase', 'Glucose-6-phosphate isomerase'),
 ('6-phosphofructokinase', 'Phosphofructokinase'),
 ('fructose-bisphosphatase', 'Fructose-bisphosphatase'),
 ('fructose-bisphosphate aldolase', 'Fructose-bisphosphate aldolase'),
 ('triose-phosphate isomerase', 'Triose-phosphate isomerase'),
 ('glyceraldehyde-3-phosphate dehydrogenase (phosphorylating)',
  'Glyceraldehyde-3-phosphate dehydrogenase'),
 ('phosphoglycerate kinase', 'Phosphoglycerate kinase'),
 ('pyruvate kinase', 'Pyruvate kinase'),
 ('phosphoglycerate mutase (2,3-diphosphoglycerate-dependent)',
  'Phosphoglycerate mutase')]

In [56]:
data

[{'map_name': 'iJO1366.Central metabolism',
  'map_id': 'c50080dc494c0686982966090cefc2de',
  'map_description': 'Central metabolism (iJO1366)\nLast Modified Mon Oct 12 2015 17:39:00 GMT-0700 (PDT)',
  'homepage': 'https://escher.github.io',
  'schema': 'https://escher.github.io/escher/jsonschema/1-0-0#'},
 {'reactions': {'2075839': {'name': 'Malate dehydrogenase',
    'bigg_id': 'MDH',
    'reversibility': True,
    'label_x': 3019.1480390625,
    'label_y': 3883.49973125,
    'gene_reaction_rule': 'b3236',
    'genes': [{'bigg_id': 'b3236', 'name': 'mdh'}],
    'metabolites': [{'coefficient': -1, 'bigg_id': 'mal__L_c'},
     {'coefficient': 1, 'bigg_id': 'h_c'},
     {'coefficient': -1, 'bigg_id': 'nad_c'},
     {'coefficient': 1, 'bigg_id': 'oaa_c'},
     {'coefficient': 1, 'bigg_id': 'nadh_c'}],
    'segments': {'675': {'from_node_id': '2076049',
      'to_node_id': '2076047',
      'b1': None,
      'b2': None},
     '676': {'from_node_id': '2076047',
      'to_node_id': '2076048'